<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Linear-optimisation-with-pulp" data-toc-modified-id="Linear-optimisation-with-pulp-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Linear optimisation with <code>pulp</code></a></span><ul class="toc-item"><li><span><a href="#Simple-linear-optimisation-model" data-toc-modified-id="Simple-linear-optimisation-model-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Simple linear optimisation model</a></span></li><li><span><a href="#Car-manufacturer" data-toc-modified-id="Car-manufacturer-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Car manufacturer</a></span></li><li><span><a href="#Blending-problem" data-toc-modified-id="Blending-problem-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Blending problem</a></span></li></ul></li><li><span><a href="#Using-pyomo-for-optimisation-models" data-toc-modified-id="Using-pyomo-for-optimisation-models-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Using <code>pyomo</code> for optimisation models</a></span></li></ul></div>

In [1]:
# Example for linear optimisation model with PULP
# Date: 2021-01-28, Author: LG

# Linear optimisation with `pulp`

`PuLP` is a python module that has a few of very useful data analysis methods. This notebook has some examples of the problems that can be solved with `pandas`.

There are `PuLP` tutorials in [here](https://coin-or.github.io/pulp/) lists most of the useful operations of `pyomo`. A good overview of examples can be found [here](https://pyomo.readthedocs.io/en/stable/bibliography.html#pyomobookii).

Here is one commented example that shows some of the functions used in the tutorials mentioned above.

## Simple linear optimisation model 

In [2]:
import pulp
my_lp_problem = pulp.LpProblem("My LP Problem", pulp.LpMaximize)
x = pulp.LpVariable('x', lowBound=0, cat='Continuous')
y = pulp.LpVariable('y', lowBound=2, cat='Continuous')

/Users/gast/anaconda3/lib/python3.6/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [3]:
# Objective function
my_lp_problem += 4 * x + 3 * y, "Z"

# Constraints
my_lp_problem += 2 * y <= 25 - x
my_lp_problem += 4 * y >= 2 * x - 8
my_lp_problem += y <= 2 * x - 5

In [4]:
my_lp_problem

My_LP_Problem:
MAXIMIZE
4*x + 3*y + 0
SUBJECT TO
_C1: x + 2 y <= 25

_C2: - 2 x + 4 y >= -8

_C3: - 2 x + y <= -5

VARIABLES
x Continuous
2 <= y Continuous

In [5]:
my_lp_problem.solve()
pulp.LpStatus[my_lp_problem.status]

'Optimal'

In [6]:
for variable in my_lp_problem.variables():
    print ('{}={}'.format(variable.name, variable.varValue))

x=14.5
y=5.25


In [7]:
print (pulp.value(my_lp_problem.objective))

73.75


## Car manufacturer 

In [8]:
# Instantiate problem class
model = pulp.LpProblem("Profit maximising problem", pulp.LpMaximize)
A = pulp.LpVariable('A', lowBound=0, cat='Integer')
B = pulp.LpVariable('B', lowBound=0, cat='Integer')

/Users/gast/anaconda3/lib/python3.6/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [9]:
# Objective function
model += 30000 * A + 45000 * B, "Profit"

# Constraints
model += 3 * A + 4 * B <= 30
model += 5 * A + 6 * B <= 60
model += 1.5 * A + 3 * B <= 21

In [10]:
model

Profit_maximising_problem:
MAXIMIZE
30000*A + 45000*B + 0
SUBJECT TO
_C1: 3 A + 4 B <= 30

_C2: 5 A + 6 B <= 60

_C3: 1.5 A + 3 B <= 21

VARIABLES
0 <= A Integer
0 <= B Integer

In [11]:
# Solve our problem
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [12]:
# Print our decision variable values
print ("Production of Car A = {}".format(A.varValue))
print ("Production of Car B = {}".format(B.varValue))

Production of Car A = 2.0
Production of Car B = 6.0


In [13]:
# Print our objective function value
print (pulp.value(model.objective))

330000.0


## Blending problem

In [14]:
# Instantiate our problem class
model = pulp.LpProblem("Cost minimising blending problem", pulp.LpMinimize)

/Users/gast/anaconda3/lib/python3.6/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [15]:
# Construct our decision variable lists
sausage_types = ['economy', 'premium']
ingredients = ['pork', 'wheat', 'starch']

In [16]:
ing_weight = pulp.LpVariable.dicts("weight kg",
    ((i, j) for i in sausage_types for j in ingredients),
    lowBound=0,
    cat='Continuous')

In [17]:
# Objective Function
model += (
    pulp.lpSum([
        4.32 * ing_weight[(i, 'pork')]
        + 2.46 * ing_weight[(i, 'wheat')]
        + 1.86 * ing_weight[(i, 'starch')]
        for i in sausage_types])
)

In [18]:
#Quantity
qt_economy=350
qt_premium=500
# Constraints
# 350 economy and 500 premium sausages at 0.05 kg
model += pulp.lpSum([ing_weight['economy', j] for j in ingredients]) == qt_economy * 0.05
model += pulp.lpSum([ing_weight['premium', j] for j in ingredients]) == qt_premium * 0.05

# Economy has >= 40% pork, premium >= 60% pork
model += ing_weight['economy', 'pork'] >= (
    0.4 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))

model += ing_weight['premium', 'pork'] >= (
    0.6 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

# Sausages must be <= 25% starch
model += ing_weight['economy', 'starch'] <= (
    0.25 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))

model += ing_weight['premium', 'starch'] <= (
    0.25 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

# We have at most 30 kg of pork, 20 kg of wheat and 17 kg of starch available
model += pulp.lpSum([ing_weight[i, 'pork'] for i in sausage_types]) <= 30
model += pulp.lpSum([ing_weight[i, 'wheat'] for i in sausage_types]) <= 20
model += pulp.lpSum([ing_weight[i, 'starch'] for i in sausage_types]) <= 17

# We have at least 23 kg of pork to use up
model += pulp.lpSum([ing_weight[i, 'pork'] for i in sausage_types]) >= 23


In [19]:
# Solve our problem
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [20]:
for var in ing_weight:
    var_value = ing_weight[var].varValue
    print ("The weight of {0} in {1} sausages is {2} kg".format(var[1], var[0], var_value))

The weight of pork in economy sausages is 7.0 kg
The weight of wheat in economy sausages is 6.125 kg
The weight of starch in economy sausages is 4.375 kg
The weight of pork in premium sausages is 16.0 kg
The weight of wheat in premium sausages is 2.75 kg
The weight of starch in premium sausages is 6.25 kg


In [14]:
total_cost = pulp.value(model.objective)

print("The total cost is € for", qt_economy, "economy sausages and",qt_premium,"premium sausages".format(round(total_cost, 2)))

The total cost is € for 350 economy sausages and 500 premium sausages


In [15]:
income = qt_economy*1 + qt_premium*2
revenue = income-total_cost
print("The income from sales (1€ for economy, 2€ for premium) is",income,"€. Therefore the revenue is:",revenue,"€")

The income from sales (1€ for economy, 2€ for premium) is 1350 €. Therefore the revenue is: 1209.045 €


# Using `pyomo` for optimisation models

`pyomo` is a python module that can be used for simple mathematical modeling techniques. <br>
There are `pyomo` tutorials in the [documentation](https://pyomo.readthedocs.io/en/stable/working_models.html) lists most of the useful application areas. <br>
An overview of sources and references can be found [here](http://benalexkeen.com/linear-programming-with-python-and-pulp/).


In [22]:

# knapsack.py
from pyomo.environ import *
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

In [23]:
model = ConcreteModel()
model.x = Var( A, within=Binary )
model.value = Objective(
    expr = sum( b[i]*model.x[i] for i in A ),
    sense = maximize )
model.weight = Constraint(
    expr = sum( w[i]*model.x[i] for i in A) <= W_max )

In [24]:
opt = SolverFactory('glpk')
x = result_obj = opt.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/s1/mk002cqs18s8j8ddy8yf00jc0000gq/T/tmp3s_lrslh.glpk.raw
 --wglp /var/folders/s1/mk002cqs18s8j8ddy8yf00jc0000gq/T/tmpp5zaiebt.glpk.glp
 --cpxlp /var/folders/s1/mk002cqs18s8j8ddy8yf00jc0000gq/T/tmpdi7c3d1m.pyomo.lp
Reading problem data from '/var/folders/s1/mk002cqs18s8j8ddy8yf00jc0000gq/T/tmpdi7c3d1m.pyomo.lp'...
/var/folders/s1/mk002cqs18s8j8ddy8yf00jc0000gq/T/tmpdi7c3d1m.pyomo.lp:28: warning: lower bound of variable 'x1' redefined
/var/folders/s1/mk002cqs18s8j8ddy8yf00jc0000gq/T/tmpdi7c3d1m.pyomo.lp:28: warning: upper bound of variable 'x1' redefined
2 rows, 5 columns, 5 non-zeros
4 integer variables, all of which are binary
32 lines were read
Writing problem data to '/var/folders/s1/mk002cqs18s8j8ddy8yf00jc0000gq/T/tmpp5zaiebt.glpk.glp'...
22 lines were written
GLPK Integer Optimizer, v4.65
2 rows, 5 columns, 5 non-zeros
4 integer variables, all of which are binary
Preprocessing...
1 

In [25]:
model.pprint()

1 Set Declarations
    x_index : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=None
        ['hammer', 'screwdriver', 'towel', 'wrench']

1 Var Declarations
    x : Size=4, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :   1.0 :     1 : False : False : Binary
        screwdriver :     0 :   1.0 :     1 : False : False : Binary
              towel :     0 :   1.0 :     1 : False : False : Binary
             wrench :     0 :   0.0 :     1 : False : False : Binary

1 Objective Declarations
    value : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 8*x[hammer] + 3*x[wrench] + 6*x[screwdriver] + 11*x[towel]

1 Constraint Declarations
    weight : Size=1, Index=None, Active=True
        Key  : Lower : Body                                                      : Upper : Active
        None :  -Inf : 5*x[hammer] + 7*x[wrench] + 4*x[screwdriver] + 3*x[t

In [26]:
model.display()

Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :   1.0 :     1 : False : False : Binary
        screwdriver :     0 :   1.0 :     1 : False : False : Binary
              towel :     0 :   1.0 :     1 : False : False : Binary
             wrench :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    value : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  25.0

  Constraints:
    weight : Size=1
        Key  : Lower : Body : Upper
        None :  None : 12.0 :  14.0


In [27]:
with open("MILP-Results.txt", "w") as f:
    model.pprint(ostream=f)
    #Ostream function
    
text = open("MILP-Results.txt", "r")

In [28]:
print(text.read()) 

1 Set Declarations
    x_index : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=None
        ['hammer', 'screwdriver', 'towel', 'wrench']

1 Var Declarations
    x : Size=4, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :   1.0 :     1 : False : False : Binary
        screwdriver :     0 :   1.0 :     1 : False : False : Binary
              towel :     0 :   1.0 :     1 : False : False : Binary
             wrench :     0 :   0.0 :     1 : False : False : Binary

1 Objective Declarations
    value : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 8*x[hammer] + 3*x[wrench] + 6*x[screwdriver] + 11*x[towel]

1 Constraint Declarations
    weight : Size=1, Index=None, Active=True
        Key  : Lower : Body                                                      : Upper : Active
        None :  -Inf : 5*x[hammer] + 7*x[wrench] + 4*x[screwdriver] + 3*x[t